# Implementando o Mecanismo de Atenção em Python sem uso de frameworks

Esse projeto tem como objetivo aprender como funciona o mecanismo de atenção em um modelo Transformer. 

Attention Is All You Need: https://arxiv.org/abs/1706.03762


In [ ]:
#!pip install -q -U torch

In [1]:
# Imports
import torch
from torch import nn
import warnings
warnings.filterwarnings('ignore')

### Classe que implementa um modelo Transformer para aprendizado de sequência usando PyTorch

`nn.Module` (nn = neural network) é a classe base fundamental de todos os modelos e camadas do PyTorch. Quase tudo que constrído no PyTorch herda dela: redes neurais completas, camadas individuais, blocos de atenção, embeddings, etc.

Camadas:

`nn.Embedding` é uma camada do PyTorch que transforma cada token representado por um número inteiro em um vetor denso de dimensão fixa. Ela funciona como uma tabela de lookup onde cada índice do vocabulário corresponde a um vetor treinável, permitindo que a rede aprenda representações contínuas e significativas para palavras ou tokens. Em vez de usar one-hot vectors enormes e esparsos, o embedding fornece vetores compactos que capturam relações semânticas e sintáticas durante o treinamento, servindo como a primeira etapa essencial em modelos de NLP como Transformers, RNNs e modelos de linguagem.

`nn.MultiheadAttention`: é uma camada do PyTorch que implementa o mecanismo de atenção multi-cabeça usado nos Transformers. Ela recebe vetores de entrada e aprende a calcular relações entre eles usando várias “cabeças” de atenção paralelas, onde cada cabeça foca em um tipo diferente de dependência entre tokens. Essa camada combina consultas, chaves e valores (Q, K, V), aplica atenção escalonada e depois concatena os resultados das cabeças. É uma camada completa, treinável e central para a arquitetura Transformer.

`nn.Sequential` com Linear → ReLU → Linear: isso também é uma camada, mais especificamente um bloco feed-forward totalmente conectado, típico de cada camada do Transformer. O nn.Sequential apenas organiza várias camadas em sequência: a primeira nn.Linear projeta o vetor para uma dimensão intermediária (às vezes maior), o nn.ReLU introduz não linearidade e a segunda nn.Linear traz de volta para a dimensão original. Esse bloco é aplicado posição



In [3]:
class Transformer(nn.Module):

    def __init__(self, vocab_size, embedding_dim, n_heads, n_layers, dropout):

        # Inicializa a classe base nn.Module
        super().__init__()

        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.n_heads = n_heads
        self.n_layers = n_layers
        self.dropout = dropout

        # Camada de embedding para converter tokens em vetores densos
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        # Define o mecanismo de atenção multi-cabeça
        self.attention = nn.MultiheadAttention(embedding_dim, n_heads, dropout=dropout)

        # Camada feed-forward para processamento adicional
        self.feed_forward = nn.Sequential(
            nn.Linear(embedding_dim, embedding_dim),
            nn.ReLU(),
            nn.Linear(embedding_dim, embedding_dim)
        )

        self.out = nn.Linear(embedding_dim, vocab_size) 

        # DEfine a camada de saída final que transformará a seuqencia de saída 
        self.out = nn.Linear(embedding_dim, vocab_size)

    def forward(self, x):

        # Aplica a camada de embedding
        x = self.embedding(x)   

        # Aplica o mecanismo de atenção multi-cabeça
        x = self.attention(x)

        # Aplica a camada feed-forward
        x = self.feed_forward(x)

        # Aplica a camada de saída final
        x = self.out(x)

        return x



Parâmetros: 


`b_size`: representa o número total de tokens que o modelo consegue reconhecer e gerar. Ele define o tamanho da tabela de embeddings e o número de classes no softmax final. Um vocabulário maior permite representar mais palavras e subpalavras, mas aumenta a memória e o custo computacional. Em resumo, o vocab_size determina o universo linguístico em que o modelo opera.

`embedding_dim`: é a dimensão dos vetores usados para representar cada token do vocabulário. Cada palavra é convertida em um vetor contínuo desse tamanho, que carrega informações semânticas e sintáticas. Dimensões maiores aumentam a capacidade expressiva do modelo, mas também exigem mais memória e computação. É um dos hiperparâmetros que mais influenciam a qualidade do modelo.

`n_heads`: é o número de cabeças de atenção utilizadas no mecanismo de Multi-Head Attention. Cada cabeça aprende a focar em padrões diferentes da sequência, como relações sintáticas ou dependências distantes. Ao usar várias cabeças em paralelo, o modelo capta múltiplos tipos de interação entre tokens ao mesmo tempo. Mais cabeças aumentam a capacidade, mas exigem que embedding_dim seja divisível pelo número de cabeças.

`n_layers`: indica quantas camadas empilhadas formam o modelo Transformer. Cada camada contém atenção multi-head, normalização e redes feed-forward, expandindo a profundidade e a capacidade de abstração. Mais camadas permitem que o modelo aprenda padrões mais complexos, porém aumentam tempo de treinamento e risco de overfitting. Esse parâmetro define o “tamanho” estrutural do modelo.

`dropout`: é a taxa de probabilidade usada para desligar aleatoriamente partes da rede durante o treinamento. Isso força o modelo a não depender de unidades específicas e melhora a generalização. Valores típicos variam entre 0.1 e 0.3, equilibrando regularização e desempenho. O dropout reduz overfitting e torna o modelo mais robusto.



In [8]:
modelo = Transformer(vocab_size = 1000, 
                     embedding_dim = 32, 
                     n_heads = 4, 
                     n_layers = 2, 
                     dropout = 0.5)

In [9]:
modelo.attention    

MultiheadAttention(
  (out_proj): NonDynamicallyQuantizableLinear(in_features=32, out_features=32, bias=True)
)